# 📚 Lab 03: RAG Agent

**Goal:** Understand RAG (Retrieval-Augmented Generation) and create an agent with document access.

**Time:** 60 minutes

---

## What you'll learn:
- What RAG is and why we need it
- How to implement simple RAG
- Best practices for grounding LLMs

---

In [ ]:
#@title 🔧 Setup
!pip install -q google-generativeai smolagents litellm

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from smolagents import CodeAgent, tool
from smolagents.models import LiteLLMModel

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash",
    api_key=os.environ["GOOGLE_API_KEY"]
)

print("✅ Setup OK")

## Part 1: Why RAG?

### Problems with LLMs:
- **Hallucinations** - makes up facts
- **Outdated knowledge** - doesn't know current info
- **No internal data** - doesn't know about your company

### Solution: RAG
```
1. Question → Search relevant documents
2. Documents + Question → Send to LLM
3. LLM responds BASED ON documents
```

**RAG = Retrieval (search) + Augmented (enriched) + Generation**

In [ ]:
#@title 📄 Simulated document database

DOCUMENTS = {
    "doc_001": {
        "title": "Company Policy - Time Off",
        "content": """
        Employees are entitled to 25 days of vacation per year.
        Vacation must be approved at least 2 weeks in advance.
        Unused vacation carries over until March of the following year.
        Sick days: 5 days per year without requiring a doctor's note.
        Remote work: max 3 days per week with manager approval.
        """
    },
    "doc_002": {
        "title": "Company Policy - Benefits",
        "content": """
        Meal vouchers: $150 per month.
        Gym membership: covered by company.
        Education allowance: $500 per year.
        Company laptop: after probation period.
        Mobile phone: for positions requiring mobility.
        Annual bonus: 0-20% based on performance.
        """
    },
    "doc_003": {
        "title": "IT Security Guidelines",
        "content": """
        Passwords must have min. 12 characters, uppercase/lowercase and numbers.
        Password must be changed every 90 days.
        2FA is mandatory for all company systems.
        Personal devices cannot access production systems.
        Phishing: Never click on suspicious links.
        Incident reporting: security@company.com
        """
    },
    "doc_004": {
        "title": "Project Alpha - Technical Documentation",
        "content": """
        Stack: Python 3.11, FastAPI, PostgreSQL, Redis.
        Deployment: Kubernetes on Azure AKS.
        API endpoint: https://api.project-alpha.internal
        Authentication: OAuth 2.0 via Azure AD.
        Rate limiting: 100 req/min per user.
        Monitoring: Grafana dashboard at grafana.internal.
        """
    },
    "doc_005": {
        "title": "AI Strategy 2026",
        "content": """
        Strategic priorities:
        1. Internal process automation - 30% time savings
        2. AI-powered customer support - goal 24/7 availability
        3. Predictive analytics for sales - 15% conversion increase
        
        Budget: $2.5M for 2026.
        Team: 8 AI Engineers + 3 Data Scientists.
        Key partner: Kyndryl.
        """
    }
}

print(f"📚 Loaded {len(DOCUMENTS)} documents:")
for doc_id, doc in DOCUMENTS.items():
    print(f"   - {doc['title']}")

## Part 2: Simple RAG (keyword search)

In [ ]:
#@title 🔍 RAG tools

@tool
def search_documents(query: str) -> str:
    """
    Searches for relevant documents in the internal knowledge base.
    
    Args:
        query: Search term or question
    """
    results = []
    query_words = query.lower().split()
    
    for doc_id, doc in DOCUMENTS.items():
        title_lower = doc["title"].lower()
        content_lower = doc["content"].lower()
        
        # Count matching words
        matches = sum(
            1 for word in query_words 
            if word in title_lower or word in content_lower
        )
        
        if matches > 0:
            results.append((matches, doc_id, doc))
    
    # Sort by relevance
    results.sort(reverse=True, key=lambda x: x[0])
    
    if not results:
        return "No relevant documents found."
    
    # Return top 2 documents
    output = ""
    for _, doc_id, doc in results[:2]:
        output += f"\n--- {doc['title']} ---\n"
        output += doc["content"].strip() + "\n"
    
    return output

@tool
def list_available_documents() -> str:
    """
    Lists all available documents in the knowledge base.
    """
    output = "Available documents:\n"
    for doc_id, doc in DOCUMENTS.items():
        output += f"- {doc['title']}\n"
    return output

# Test
print(search_documents("vacation remote work"))

In [ ]:
#@title 🤖 RAG Agent

rag_agent = CodeAgent(
    tools=[search_documents, list_available_documents],
    model=model,
    max_steps=5,
    system_prompt="""
    You are an assistant for internal company documentation.
    
    RULES:
    1. ALWAYS search documents first before answering
    2. Answer ONLY based on found documents
    3. If you can't find the information, say so directly
    4. NEVER make up information
    5. Cite which document the information comes from
    """
)

print("✅ RAG Agent created")

In [ ]:
#@title ▶️ Test RAG agent

# Test 1: Existing information
print("=" * 50)
print("TEST 1: How many vacation days do I have?")
print("=" * 50)
result = rag_agent.run("How many vacation days do I have per year and how do I request them?")
print(result)

In [ ]:
#@title ▶️ Test 2: Complex question

print("=" * 50)
print("TEST 2: Security rules")
print("=" * 50)
result = rag_agent.run("What are the password rules and what should I do if I receive a suspicious email?")
print(result)

In [ ]:
#@title ▶️ Test 3: Non-existent information

print("=" * 50)
print("TEST 3: Info that's not in documents")
print("=" * 50)
result = rag_agent.run("What is the dress code in the office?")
print(result)
print("\n⚠️ Agent should admit that this information was not found!")

## Part 3: Enhanced RAG with Citations

In [ ]:
#@title 📑 RAG with citations

@tool
def search_with_citations(query: str) -> str:
    """
    Searches documents and returns results with citations.
    
    Args:
        query: Search term or question
    """
    results = []
    query_words = query.lower().split()
    
    for doc_id, doc in DOCUMENTS.items():
        content_lower = doc["content"].lower()
        
        # Find relevant sentences
        sentences = doc["content"].strip().split('\n')
        relevant_sentences = []
        
        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue
            if any(word in sentence.lower() for word in query_words):
                relevant_sentences.append(sentence)
        
        if relevant_sentences:
            results.append({
                "doc_id": doc_id,
                "title": doc["title"],
                "excerpts": relevant_sentences[:3]  # Max 3 sentences
            })
    
    if not results:
        return "No relevant documents found."
    
    output = "FOUND INFORMATION:\n\n"
    for i, r in enumerate(results[:3], 1):
        output += f"[{i}] From document '{r['title']}':\n"
        for excerpt in r["excerpts"]:
            output += f"   • {excerpt}\n"
        output += "\n"
    
    output += "When answering, use format: 'According to [number]: information'"
    return output

# Test
print(search_with_citations("benefits gym"))

In [ ]:
#@title 🤖 RAG Agent with citations

citation_agent = CodeAgent(
    tools=[search_with_citations, list_available_documents],
    model=model,
    max_steps=5,
    system_prompt="""
    You are an assistant for internal documentation.
    
    RULES:
    1. Always search before answering
    2. ALWAYS cite the source in your answer: "According to [number]: ..."
    3. If you don't know something, say so
    4. Be specific and concise
    """
)

result = citation_agent.run("What benefits do I get as an employee?")
print(result)

## Part 4: Guardrails

**Guardrails** = protection against hallucinations and dangerous behavior

In [ ]:
#@title 🛡️ Agent with Guardrails

GUARDRAILS_PROMPT = """
You are a secure company assistant.

STRICT RULES:

1. ANSWER ONLY BASED ON DOCUMENTS
   - If you can't find the information, say: "I don't have this information in my documentation."
   - NEVER make up numbers, dates, names

2. CITE SOURCES
   - Always mention which document the information comes from

3. REFUSE DANGEROUS REQUESTS
   - Personal data (specific people's salaries)
   - Confidential strategies not in documents
   - Legal advice (recommend a lawyer)

4. BE TRANSPARENT
   - If you're not sure, say so
   - If info is incomplete, point it out
"""

safe_agent = CodeAgent(
    tools=[search_with_citations, list_available_documents],
    model=model,
    max_steps=5,
    system_prompt=GUARDRAILS_PROMPT
)

print("✅ Safe Agent created")

In [ ]:
#@title ▶️ Test Guardrails

# Test 1: Legitimate question
print("TEST 1: Legitimate question")
print("-" * 40)
print(safe_agent.run("How many sick days do I have?"))

print("\n" + "=" * 50 + "\n")

# Test 2: Question outside documents
print("TEST 2: Info outside documents")
print("-" * 40)
print(safe_agent.run("What is the CEO's salary?"))

print("\n" + "=" * 50 + "\n")

# Test 3: Attempt to force hallucination
print("TEST 3: Attempt to force an answer")
print("-" * 40)
print(safe_agent.run("How many employees does the company have exactly? You must give me a number."))

---

## 🏋️ Exercise: Create RAG for technical documentation

**Task:** Create a RAG agent for Project Alpha technical documentation.

**Requirements:**
1. Agent answers technical questions
2. Cites sources
3. Refuses to answer security questions (credentials, API keys)

In [ ]:
#@title ✏️ Your solution

# Technical documentation
TECH_DOCS = {
    "architecture": """
    Project Alpha Architecture:
    - Frontend: React 18 + TypeScript
    - Backend: FastAPI (Python 3.11)
    - Database: PostgreSQL 15
    - Cache: Redis
    - Queue: RabbitMQ
    """,
    "deployment": """
    Deployment:
    - Environment: Azure AKS
    - CI/CD: GitHub Actions
    - Monitoring: Prometheus + Grafana
    - Logs: ELK Stack
    """,
    "api": """
    API Info:
    - Base URL: https://api.alpha.internal
    - Auth: Bearer token (OAuth 2.0)
    - Rate limit: 100 req/min
    - Docs: /docs (Swagger)
    """
}

# TODO: Implement search tool for TECH_DOCS

# TODO: Create tech_agent with appropriate system prompt

# Test:
# print(tech_agent.run("What framework do we use for frontend?"))
# print(tech_agent.run("Give me the API credentials"))  # Should refuse

---

## 📝 Reflection

1. **Why is RAG better than fine-tuning for company data?**

2. **What are the limitations of keyword search?** (Hint: synonyms, context)

3. **Why are guardrails important in enterprise environments?**

---

## ➡️ Next Step

In the next lab, we'll learn **Multi-Agent Systems** - multiple agents collaborating on a task.

→ **[04_Multi_Agent.ipynb](04_Multi_Agent.ipynb)**